# Convolutional Neural Network 
## Lung Nodule detection from CT scans

Import Modules:

In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob
import SimpleITK as sitk #a library for handling and processing medical images
from PIL import Image #a library for adding image processing capabilities to your Python interpreter
from scipy.misc import imread

### Some other features 
%matplotlib inline
from IPython.display import clear_output
pd.options.mode.chained_assignment = None

**Import Annotations:**

In [21]:
!pwd
annotations = pd.read_csv('/Users/raziel/Documents/CNN-for-detecting-Lung-Nodules/Datasets/annotations.csv')
candidates = pd.read_csv('/Users/raziel/Documents/CNN-for-detecting-Lung-Nodules/Datasets/candidates.csv')

/Users/raziel/Documents/CNN-for-detecting-Lung-Nodules


In [32]:
annotations.head()

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150


In [30]:
candidates.head()

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-56.08,-67.85,-311.92,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,53.21,-244.41,-245.17,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.66,-121.80,-286.62,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-33.66,-72.75,-308.41,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-32.25,-85.36,-362.51,0


**Exploration of data**

In [41]:
print('Number of rows in annotation: '+ str(len(annotations)))
print('Number of rows in candidates: '+ str(len(candidates)))

### Kind of classes in candidates
print( 'Number of negative in candidates: '  + str(len(candidates[candidates['class'] == 0])) )
print( 'Number of positive candidates: '  + str(len(candidates[candidates['class'] == 1])) )

Number of rows in annotation: 1186
Number of rows in candidates: 551065
Number of negative in candidates: 549714
Number of positive candidates: 1351


In [40]:
import multiprocessing
num_cores = multiprocessing.cpu_count()
print(num_cores)

4


### Classes are very heaviliy unbalanced, 0.25% are positive

The best way to move forward will be to undersample the negative class and then augment the positive class heaviliy to balance out the samples.